# Create firm-specific features and macro-economic features

In [1]:
# Install WRDS and SASPY modules

# pip install wrds
# pip install saspy

## WRDS connection

In [1]:
###################
# Connect to WRDS #
###################

import wrds
conn=wrds.Connection()

# Check connection
conn

Enter your WRDS username [lijunandkahshin]:ref2018a
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
Loading library list...
Done


## COMP annual fundamentals

### Create features

In [2]:
%%time
# Compustat Annual Fundamentals

import pandas as pd
import numpy as np
import datetime as dt
import psycopg2 
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats


###################
# Compustat Block #
###################
comp = conn.raw_sql("""
                    SELECT 
                        /*header info*/
                        cusip as cnum, c.gvkey, datadate, fyear,
                        c.cik, SUBSTR(sic,1,2) AS sic2, sic, naics,

                        /*firm variables*/
                        /*income statement*/
                        sale, revt, cogs, xsga, dp, xrd, xad, ib,
                        ebitda, ebit, nopi, spi, pi, txp, ni, txfed, txfo, txt, xint,

                        /*CF statement and others*/
                        capx, oancf, dvt, ob, gdwlia, gdwlip, gwo,

                        /*assets*/
                        rect, act, che, ppegt, invt, at, aco, intan, ao,
                        ppent, gdwl, fatb, fatl,

                        /*liabilities*/
                        lct, dlc, dltt, lt, dm, dcvt, cshrc, dcpstk,
                        pstk, ap, lco, lo, drc, drlt, txdi,

                        /*equity and other*/
                        ceq, scstkc, emp, csho,

                        /*market*/
                        ABS(prcc_f) as prcc_f

                    FROM comp.company as c, comp.funda as f

                    WHERE f.gvkey = c.gvkey

                    /*require some reasonable amount of information*/

                    /*get consolidated, standardized, industrial format statements*/
                    AND f.indfmt='INDL' 
                    AND f.datafmt='STD' 
                    AND f.popsrc='D' 
                    AND f.consol='C'
                    AND datadate >= '01/01/1975'
                    """)

# Require some reasonable amount of information, drop rows with missing 
#  Assets - Total, Price Close - Annual - Fiscal, Net Income (Loss)
comp.dropna(subset = ['at', 'prcc_f', 'ni'], inplace = True)

# Create mve_f: "Common Shares Outstanding" x "Price Close - Annual - Fiscal"
comp['mve_f'] = comp['csho'] * comp['prcc_f']

# Create substring of CUSIP, taking first 6 characters
comp['cnum'] = comp['cnum'].str[0:6]

# Change dtypes
comp[['gvkey']] = comp[['gvkey']].astype(int)
comp['datadate'] = pd.to_datetime(comp['datadate'])
comp[['cik','sic','sic2','naics','gwo']] = comp[
    ['cik','sic','sic2','naics','gwo']
].astype(float)

# Drop duplicates
comp.drop_duplicates(inplace = True)

# Sort by gvkey & date, then create "count" variable for gvkey
comp.sort_values(['gvkey','datadate'], inplace = True)
comp.reset_index(drop=True, inplace = True)
comp['count'] = comp.groupby('gvkey').cumcount() + 1


# Create "dr" (Deferred Revenue) variable, by using
#   "drc" (Deferred Revenue - Current) and "drtl" (Deferred Revenue - Long-term)
def create_dr(row):
    if pd.notnull(row['drc']) and pd.notnull(row['drlt']):
        return row['drc'] + row['drlt']
    if pd.notnull(row['drc']) and pd.isnull(row['drlt']):
        return row['drc']
    if pd.isnull(row['drc']) and pd.notnull(row['drlt']):
        return row['drlt']
    else:
        return np.nan  
comp['dr'] = comp.apply(lambda x: create_dr(x), axis = 1)

# Create "dc" (Convertible Debt) variable, by using
#   "dcvt" (Debt - Convertible), "dcpstk" (Convertible Debt and Preferred Stock), and
#   'pstk' (Preferred/Preference Stock (Capital) - Total)
def create_dc(row):
    if (
        (
            pd.isnull(row['dcvt']) and pd.notnull(row['dcpstk'])
        ) and pd.notnull(row['pstk'])
    ) and row['dcpstk'] > row['pstk']:
        return row['dcpstk'] - row['pstk']
    
    if (
        pd.isnull(row['dcvt']) and pd.notnull(row['dcpstk'])
    ) and pd.isnull(row['dcvt']):
        return row['dcpstk']
    if pd.notnull(row['dcvt']):
        return row['dcvt']
    else:
        return np.nan 
comp['dc'] = comp.apply(lambda x: create_dc(x), axis = 1)

# Create "xint0" variable
comp['xint0'] = comp['xint'].map(lambda x: 0 if np.isnan(x) else x)

# Create "xsga0" variable
comp['xsga0'] = comp['xsga'].map(lambda x: 0 if np.isnan(x) else x) ## GHZ(2016) has an error in the SAS code

### Create first pass variable - just using annual Compustat variables ###
# Create lagged variables
to_lag1 = ['at', 'lt', 'act', 'che', 'lct', 'dlc', 'txp', 'invt', 'emp', 
          'sale', 'rect', 'cogs', 'xsga', 'dp', 'ppent', 'xad', 'ppegt',
         'ceq', 'capx', 'gdwl', 'dvt', 'ob', 'aco', 'intan', 'ao', 'ap',
          'lco', 'lo', 'dr', 'xrd', 'ni', 'dltt', 'csho', 'ib']
to_lag2 = ['capx', 'at']
for i in to_lag1:
    comp[f'{i}_lag1'] = comp.groupby('gvkey')[i].shift(1)
for i in to_lag2:
    comp[f'{i}_lag2'] = comp.groupby('gvkey')[i].shift(2)

# Book-to-market ratio (Rosenberg, Reid, and Lanstein, 1985, JPM)
comp['bm'] = comp['ceq'] / comp['mve_f'].replace({ 0 : np.nan })

# Earnings-price (Basy, 1977, JF)
comp['ep'] = comp['ib'] / comp['mve_f'].replace({ 0 : np.nan })

# Fiscal year-end market capitalization plus long-term debt (dltt) minus total assets (at) divided by cash and equivalents (che) (Chandrashekar and Rao, 2009, WP) 
comp['cashpr'] = ((comp['mve_f'] + comp['dltt'] - comp['at']) \
                  / comp['che'].replace({ 0 : np.nan }))

# Dividend yield (Litzenberger and Ramaswamy, 1982, JF)
comp['dy'] = comp['dvt'] / comp['mve_f'].replace({ 0 : np.nan })

# Leverage (Bhandari, 1988, JF)
comp['lev'] = comp['lt'] / comp['mve_f'].replace({ 0 : np.nan })

# Sales (Barbee, Mukherji, and Raines, 1996, FAJ)
comp['sp'] = comp['sale'] / comp['mve_f'].replace({ 0 : np.nan })

# Return on invested capital (Brown and Rowe, 2007, WP)
comp['roic'] = (comp['ebit'] - comp['nopi']) \
                /(comp['ceq'] + comp['lt'] - comp['che']).replace({ 0 : np.nan })

# R&D expense divided by sales and market cap (Guo, Lev, and Shi, 2006, JBFA)
comp['rd_sale'] = comp['xrd'] / comp['sale'].replace({ 0 : np.nan })
comp['rd_mve'] = comp['xrd'] / comp['mve_f'].replace({ 0 : np.nan })

# Change in total assets (Cooper, Gulen, and Schill, 2008, JF)
comp['agr'] = (comp['at'] / comp['at_lag1'].replace({ 0 : np.nan })) - 1

# Gross profitability premium (Novy-Marx, 2013, JFE)
comp['gma'] = (comp['revt'] - comp['cogs']) / comp['at_lag1'].replace({ 0 : np.nan })

# Change in outstanding shares (Pontiff and Woodgate, 2008, JF)
comp['chcsho'] = (comp['csho'] / comp['csho_lag1'].replace({ 0 : np.nan })) -1

# Change in total liabilities (Richardson et al, 2005, JAE)
comp['lgr'] = (comp['lt'] / comp['lt_lag1'].replace({ 0 : np.nan })) -1

# Accrual anomaly (Sloan, 1996, TAR) ## Typo in GHZ(1996) code, minus sign is wrong for "dp" variable
def create_acc(row):
    if (row['at'] + row['at_lag1']) == 0:
        return np.nan
    if pd.notnull(row['oancf']):
        return (row['ib'] - row['oancf']) \
    / ((row['at'] + row['at_lag1'])/2) #.replace({ 0 : np.nan })
    else:
        return (
            (# Change in current assets minus Change in cash
                (row['act'] - row['act_lag1']) - (row['che'] - row['che_lag1'])
            ) \
            - (# Change in current liabilities minus Change in debt minus Change in tax payable
                (row['lct'] - row['lct_lag1']) \
                - (row['dlc'] - row['dlc_lag1']) \
                - (row['txp'] - row['txp_lag1'])
            ) \
            - row['dp'] # minus depreciation and amortiation expense
        ) / ((row['at'] + row['at_lag1'])/2) #.replace({ 0 : np.nan })

comp['acc'] = comp.apply(lambda x: create_acc(x), axis = 1)
    
# Percent accrual (Hafzalla, Lundholm, and Van Winkle, 2011, TAR)  
def create_pctacc(row):
    if row['ib'] != 0 and pd.notnull(row['oancf']):
        return (row['ib'] - row['oancf']) / abs(row['ib']) 
    if row['ib'] == 0 and pd.notnull(row['oancf']):
        return (row['ib'] - row['oancf']) / 0.01
    if row['ib'] != 0 and pd.isnull(row['oancf']):  
        return (
            (
            (row['act'] - row['act_lag1']) - (row['che'] - row['che_lag1'])
            ) \
            - (
                (row['lct'] - row['lct_lag1']) \
                - (row['dlc'] - row['dlc_lag1']) \
                - (row['txp'] - row['txp_lag1'])
            ) \
            - row['dp'] ) / abs(row['ib']) ## GHZ(2016) code is wrong for 'dp' sign
    if row['ib'] == 0 and pd.isnull(row['oancf']):
        return (
            (
            (row['act'] - row['act_lag1']) - (row['che'] - row['che_lag1'])
            ) \
            - (
                (row['lct'] - row['lct_lag1']) \
                - (row['dlc'] - row['dlc_lag1']) \
                - (row['txp'] - row['txp_lag1'])
            ) \
            - row['dp'] ) / 0.01          ## GHZ(2016) code is wrong for 'dp' sign
    
comp['pctacc'] = comp.apply(lambda x: create_pctacc(x), axis = 1)

# Operating cash flows divided by mkt cap (Desai, Rajgopal, and Venkatachalam, 2004, TAR)
def create_cfp(row):
    if row['mve_f'] == 0:
        return np.nan
    if pd.notnull(row['oancf']):
        return row['oancf'] / row['mve_f']
    else:
        return (
            row['ib'] - (
                ((row['act'] - row['act_lag1']) - (row['che'] - row['che_lag1'])
                ) \
                - \
                ((row['lct'] - row['lct_lag1']) \
                 - (row['dlc'] - row['dlc_lag1']) \
                 - (row['txp'] - row['txp_lag1'])
                ) \
                - row['dp']               ## GHZ(2016) code is wrong for 'dp' sign
            )
        ) / row['mve_f']

comp['cfp'] = comp.apply(lambda x: create_cfp(x), axis = 1)

# Absolute value of "acc" (Bandyopadhyay, Huang, and Wirjanto, 2010, WP) 
comp['absacc'] = abs(comp['acc'])

# Years since first Compustat coverage (Jiang, Lee, and Zhang, 2005, RAS)
comp['age'] = comp['count']

# Change in inventory scaled by assets (Jiang, Lee, and Zhang, 2002, RAS)
comp['chinv'] = (comp['invt'] - comp['invt_lag1']) \
                / ((comp['at'] + comp['at_lag1'])/2).replace({ 0 : np.nan })


# Special items "spi" indicator
comp['spii'] = comp['spi'].map(lambda x: 1 if (x != 0 and pd.notnull(x)) else 0)

# Special items over total assets
comp['spi'] = comp['spi'] / ((comp['at'] + comp['at_lag1'])/2).replace({ 0 : np.nan })

# Cash flow divided by asset
def create_cf(row):
    if (row['at'] + row['at_lag1']) == 0:
        return np.nan
    if pd.notnull(row['oancf']):
        return row['oancf'] / ((row['at'] + row['at_lag1'])/2)
    else:
        return (
            row['ib'] - (
                ((row['act'] - row['act_lag1']) - (row['che'] - row['che_lag1'])
                ) \
                - \
                ((row['lct'] - row['lct_lag1']) \
                 - (row['dlc'] - row['dlc_lag1']) \
                 - (row['txp'] - row['txp_lag1'])
                ) \
                - row['dp']               ## GHZ(2016) code is wrong for 'dp' sign
            )
        ) / ((row['at'] + row['at_lag1'])/2)

comp['cf'] = comp.apply(lambda x: create_cf(x), axis = 1)

# Percent change in employees (Bazdresch, Belo, and Lin, 2014, JPE)
def create_hire(row):
    if pd.isnull(row['emp']) or pd.isnull(row['emp_lag1']):
        return 0
    if row['emp_lag1'] == 0:
        return np.nan
    else:
        return (row['emp'] - row['emp_lag1']) / row['emp_lag1']
comp['hire'] = comp.apply(lambda x: create_hire(x), axis = 1)

# Percent change in sales (Lakonishok, Shleifer, and Vishny, 1994, JF)
comp['sgr'] = (comp['sale'] / comp['sale_lag1'].replace({ 0 : np.nan })) - 1

# Change in income/sales (Soliman, 2008, TAR)
comp['chpm'] = (comp['ib'] / comp['sale'].replace({ 0 : np.nan })) \
                - (comp['ib_lag1'] / comp['sale_lag1'].replace({ 0 : np.nan }))

# Change in sales/total assets (Soliman, 2008, TAR)
comp['chato'] = (comp['sale'] / ((comp['at'] + comp['at_lag1'])/2).replace({ 0 : np.nan })) \
                - (comp['sale_lag1'] / ((comp['at_lag1'] + comp['at_lag2'])/2).replace({ 0 : np.nan }))

# Percent change in sales minus Percent change in inventory (Abarbanell and Bushee, 1998, TAR)
comp['pchsale_pchinvt'] = ((comp['sale'] / comp['sale_lag1'].replace({ 0 : np.nan })) -1) \
                            -((comp['invt'] / comp['invt_lag1'].replace({ 0 : np.nan })) -1)

# Percent change in sales minus Percent change in receivables (Abarbanell and Bushee, 1998, TAR)
comp['pchsale_pchrect'] = ((comp['sale'] / comp['sale_lag1'].replace({ 0 : np.nan })) -1) \
                            -((comp['rect'] / comp['rect_lag1'].replace({ 0 : np.nan })) -1)

# Percent change in gross margin minus Percent change in sales (Abarbanell and Bushee, 1998, TAR)
comp['pchgm_pchsale'] = (
    (comp['sale'] - comp['cogs']) \
    / (comp['sale_lag1'] - comp['cogs_lag1']).replace({ 0 : np.nan }) -1
) - (comp['sale'] / comp['sale_lag1'].replace({ 0 : np.nan }) - 1)

# Percent change in sales minus Percent change in SG&A (Abarbanell and Bushee, 1998, TAR)
comp['pchsale_pchxsga'] = ((comp['sale'] / comp['sale_lag1'].replace({ 0 : np.nan })) -1) \
                            -((comp['xsga'] / comp['xsga_lag1'].replace({ 0 : np.nan })) -1)

# Depreciation divided by PP&E (Holthausen and Larcker, 1992, JAE)
comp['depr'] = comp['dp'] /comp['ppent'].replace({ 0 : np.nan })

# Percent change in "depr" (Holthausen and Larcker, 1992, JAE)
comp['pchdepr'] = (
    (comp['dp'] / comp['ppent'].replace({ 0 : np.nan })) \
    / (comp['dp_lag1'] / comp['ppent_lag1'].replace({ 0 : np.nan })).replace({ 0 : np.nan }) \
) - 1

# Change in advertising expense
comp['chadv'] = np.log(1+comp['xad']) - np.log((1+comp['xad_lag1']))

# Investment factor (Chen and Zhang, 2010, JF) ## GHZ(2016) fixed extreme values, e.g. gvkey 146017, yr 2021 vs 2020 vs 2019
def create_invest(row):
    if row['at_lag1'] == 0:
        return np.nan
    if pd.notnull(row['ppegt']):
        return (
            (row['ppegt'] - row['ppegt_lag1']) \
            + (row['invt'] - row['invt_lag1'])
        ) / row['at_lag1']
    else:
        return (
            (row['ppent'] - row['ppent_lag1']) \
            + (row['invt'] - row['invt_lag1'])
        ) / row['at_lag1']    
comp['invest'] = comp.apply(lambda x: create_invest(x), axis = 1)    

# Percent change in book value of equity (Richardson et al., 2005, JAE)
comp['egr'] = (comp['ceq'] / comp['ceq_lag1'].replace({ 0 : np.nan })) -1 

# Fix missing capex
def create_capx(row):
    if pd.isnull(row['capx']) and row['count'] >= 2:
        return row['ppent'] - row['ppent_lag1']
    else:
        return row['capx']
comp['capx'] = comp.apply(lambda x: create_capx(x), axis = 1)  

# Percent change in capex (Abarbanell and Bushee, 1998, TAR) ## GHZ(2016) has too many extreme negative outliers such that the global mean is negative, probably bug in code
comp['capx_lag1'] = comp.groupby('gvkey')['capx'].shift(1)
comp['pchcapx'] = (comp['capx'] / comp['capx_lag1'].replace({ 0 : np.nan })) -1

# Percent change in capex from year t-2 to year t (Anderson and Garcia-Feijoo, 2006, JF) ## GHZ(2016) has too many extreme negative outliers such that the global mean is negative, probably bug in code
comp['capx_lag2'] = comp.groupby('gvkey')['capx'].shift(2)
comp['grcapx'] = (comp['capx'] / comp['capx_lag2'].replace({ 0 : np.nan })) -1

# Percent change in goodwill
comp['grGW'] = (comp['gdwl'] / comp['gdwl_lag1'].replace({ 0 : np.nan })) - 1 
def create_grGW(row):
    if pd.isnull(row['gdwl']) or row['gdwl'] == 0:
        return 0
    if (row['gdwl'] != 0 and pd.notnull(row['gdwl'])) and pd.isnull(row['grGW']):
        return 1
    else:
        return row['grGW']
comp['grGW'] = comp.apply(lambda x: create_grGW(x), axis = 1)
def create_woGW(row):
    if (
        (pd.notnull(row['gdwlia']) and row['gdwlia'] != 0) or \
        (pd.notnull(row['gdwlip']) and row['gdwlip'] != 0)
    ) or (pd.notnull(row['gwo']) and row['gwo'] != 0):
        return 1
    else:
        return 0
comp['woGW'] = comp.apply(lambda x: create_woGW(x), axis = 1)

# Tangibility of assets (Almeida and Campello, 2007, RFS)
comp['tang'] = (
    comp['che'] + comp['rect']*0.715 + \
    comp['invt']*0.547 + comp['ppent']*0.535
)/comp['at'].replace({ 0 : np.nan })

# Indicator for sin industry (Hong & Kacperczyk, 2009, JFE)
def create_sin(row):
    if (
        ((2100<=row['sic']<=2199) or (2080<=row['sic']<=2085)) or \
        (row['naics'] in [7132, 71312, 713210, 71329, 713290, 72112, 721120])
    ):
        return 1
    else:
        return 0
comp['sin'] = comp.apply(lambda x: create_sin(x), axis = 1)

# Impute missing 'Current Assets - Total'
def create_act(row):
    if pd.isnull(row['act']):
        return row['che'] + row['rect'] + row['invt']
    else:
        return row['act']
comp['act'] = comp.apply(lambda x: create_act(x), axis = 1)

# Impute missing 'Current Liabilities - Total'
def create_lct(row):
    if pd.isnull(row['lct']):
        return row['ap']
    else:
        return row['lct']
comp['lct'] = comp.apply(lambda x: create_lct(x), axis = 1)

# Current assets / current liabilities (Ou and Penman, 1989, JAE)
comp['currat'] = comp['act'] / comp['lct'].replace({ 0 : np.nan })

# Percent change in 'currat' (Ou and Penman, 1989, JAE)
comp['pchcurrat'] = (
    (comp['act'] / comp['lct'].replace({ 0 : np.nan })) / \
    (comp['act_lag1'] / comp['lct_lag1'].replace({ 0 : np.nan })).replace({ 0 : np.nan })
) - 1

# Direction of one-year-ahead changes (Ou and Penman, 1989, JAE)
comp['quick'] = (comp['act'] - comp['invt']) / comp['lct'].replace({ 0 : np.nan })

# Percent change in 'quick' (Ou and Penman, 1989, JAE)  ## GHZ(2016) does not fully match mine
comp['pchquick'] = (
    ((comp['act'] - comp['invt']) / comp['lct'].replace({ 0 : np.nan })) / \
    ((comp['act_lag1'] - comp['invt_lag1']) / comp['lct_lag1'].replace({ 0 : np.nan })).replace({ 0 : np.nan })
) - 1


# Annual sales divided by cash and cash equivalents (Ou and Penman, 1989, JAE)
comp['salecash'] = comp['sale'] / comp['che'].replace({ 0 : np.nan })
# Annual sales divided by cash and cash equivalents (Ou and Penman, 1989, JAE)
comp['salerec'] = comp['sale'] / comp['rect'].replace({ 0 : np.nan })
# Annual sales divided by accounts receivable  (Ou and Penman, 1989, JAE)
comp['saleinv'] = comp['sale'] / comp['invt'].replace({ 0 : np.nan })
# Percent change in 'saleinv' (Ou and Penman, 1989, JAE)
comp['pchsaleinv'] = (
    (comp['sale'] / comp['invt'].replace({ 0 : np.nan })) / \
    (comp['sale_lag1'] / comp['invt_lag1'].replace({ 0 : np.nan })).replace({ 0 : np.nan })
) -1 

# Cash debt (Chandrashekar and Rao, 2009, WP)
comp['cashdebt'] = (comp['ib'] + comp['dp']) / ((comp['lt'] + comp['lt_lag1'])/2).replace({ 0 : np.nan })

# Buildings and capitalized leases divided by gross PP&E (Tuzel, 2010, RFS)
def create_realestate(row):
    if row['ppegt'] == 0:
        return np.nan
    if pd.notnull(row['ppegt']):
        return (row['fatb'] + row['fatl']) / row['ppegt']
    if row['ppent'] == 0:
        return np.nan
    else:
        return (row['fatb'] + row['fatl']) / row['ppent']
comp['realestate'] = comp.apply(lambda x: create_realestate(x), axis = 1)

# Indicator variables for dividend payment (Michaely, Thaler, and Womack, 1995, JF)
def create_divi(row):
    if (pd.notnull(row['dvt']) and row['dvt']>0) and \
    (row['dvt_lag1'] == 0 or pd.isnull(row['dvt_lag1'])):
        return 1
    else:
        return 0
comp['divi'] = comp.apply(lambda x: create_divi(x), axis = 1)
def create_divo(row):
    if (pd.isnull(row['dvt']) or row['dvt'] ==0) and \
    (row['dvt_lag1']>0 and pd.notnull(row['dvt_lag1'])):
        return 1
    else:
        return 0
comp['divo'] = comp.apply(lambda x: create_divo(x), axis = 1)  

# Order backlog
comp['obklg'] = comp['ob'] / ((comp['at'] + comp['at_lag1'])/2).replace({ 0 : np.nan })
comp['chobklg'] = (comp['ob'] - comp['ob_lag1']) / \
                    ((comp['at'] + comp['at_lag1'])/2).replace({ 0 : np.nan })

# Total liability scaled secured debt (Valta, 2016, JFQA)
def create_securedind(row):
    if (pd.notnull(row['dm']) and row['dm'] != 0):
        return 1
    else:
        return 0
comp['securedind'] = comp.apply(lambda x: create_securedind(x), axis = 1)
comp['secured'] = comp['dm'] / comp['dltt'].replace({ 0 : np.nan })

# Convertible debt obligations  (Valta, 2016, JFQA)
def create_convind(row):
    if (pd.notnull(row['dc']) and row['dc'] != 0) or \
    (pd.notnull(row['cshrc']) and row['cshrc'] != 0):
        return 1
    else:
        return 0
comp['convind'] = comp.apply(lambda x: create_convind(x), axis = 1)
comp['conv'] = comp['dc'] / comp['dltt'].replace({ 0 : np.nan })

# Growth in long-term net operating assets (Fairfield, Whisenant, and Yohn, 2003, TAR)
comp['grltnoa'] = (
    (
        comp['rect'] + comp['invt'] + comp['ppent'] + \
        comp['aco'] + comp['intan'] + comp['ao'] - \
        comp['ap'] - comp['lco'] - comp['lo']
    ) - \
    (
        comp['rect_lag1'] + comp['invt_lag1'] + comp['ppent_lag1'] + \
        comp['aco_lag1'] + comp['intan_lag1'] + comp['ao_lag1'] - \
        comp['ap_lag1'] - comp['lco_lag1'] - comp['lo_lag1']
    ) - \
    (
        comp['rect'] - comp['rect_lag1'] + comp['invt'] - \
        comp['invt_lag1'] + comp['aco'] - comp['aco_lag1'] - \
        (comp['ap'] - comp['ap_lag1'] + comp['lco'] - comp['lco_lag1']) - \
        comp['dp'] 
    )
) / ((comp['at'] + comp['at_lag1'])/2).replace({ 0 : np.nan })

# Change in deferred revenue
comp['chdrc'] = (comp['dr'] - comp['dr_lag1']) / \
                ((comp['at'] + comp['at_lag1'])/2).replace({ 0 : np.nan })

# R&D indicator (Eberhart, Maxwell, and Siddique, 2004, JF) ## GHZ(2016)'s code has a bracketing issue, results very different
def create_rd(row):
    if (row['at'] == 0 or row['at_lag1'] == 0) \
    or (row['xrd_lag1'] == 0):
        return np.nan
    if (
        (row['xrd'] / row['at']) \
        / (row['xrd_lag1'] / row['at_lag1']) /
        - 1  
    ) > 0.5:
        return 1
    else:
        return 0
comp['rd'] = comp.apply(lambda x: create_rd(x), axis = 1)

# R&D bias
comp['rdbias'] = (comp['xrd'] / comp['xrd_lag1'].replace({ 0 : np.nan })) \
                    - 1 - (comp['ib'] / comp['ceq_lag1'].replace({ 0 : np.nan }))

# ROE
comp['roe'] = comp['ib'] / comp['ceq_lag1'].replace({ 0 : np.nan })

# Operating profit (Fama and French, 2015, JFE) 
comp['operprof'] = (comp['revt'] - comp['cogs'] - comp['xsga0'] - comp['xint0']) \
                    / comp['ceq_lag1'].replace({ 0 : np.nan })

# Fundamental health score (Piotroski, 2000, JAR)  ## GHZ(2016) code error, assumes div0 in denominator == 0 instead of NaN, 20Mc
# Sum of 9 indicator variables
comp['ind_1'] = comp['ni'] > 0 
comp['ind_2'] = comp['oancf'] > 0 
comp['ind_3'] = comp['ni']/comp['at'].replace({ 0 : np.nan }) > \
                comp['ni_lag1']/comp['at_lag1'].replace({ 0 : np.nan })
comp['ind_4'] = comp['oancf'] > comp['ni']
comp['ind_5'] = comp['dltt']/comp['at'].replace({ 0 : np.nan }) < \
                comp['dltt_lag1']/comp['at_lag1'].replace({ 0 : np.nan })
comp['ind_6'] = comp['act']/comp['lct'].replace({ 0 : np.nan }) > \
                comp['act_lag1']/comp['lct_lag1'].replace({ 0 : np.nan })
comp['ind_7'] = (comp['sale'] - comp['cogs'])/comp['sale'].replace({ 0 : np.nan }) > \
                (comp['sale_lag1'] - comp['cogs_lag1'])/comp['sale_lag1'].replace({ 0 : np.nan })
comp['ind_8'] = comp['sale']/comp['at'].replace({ 0 : np.nan }) > \
                comp['sale_lag1']/comp['at_lag1'].replace({ 0 : np.nan })
comp['ind_9'] = comp['scstkc'] == 0
comp['ps'] = comp[['ind_1','ind_2', 'ind_3', 'ind_4', 'ind_5', 
                   'ind_6', 'ind_7', 'ind_8', 'ind_9']].sum(axis=1)
comp.drop(columns = ['ind_1','ind_2', 'ind_3', 'ind_4', 'ind_5', 
                   'ind_6', 'ind_7', 'ind_8', 'ind_9'], inplace = True)

# Tax income (Lev and Nissim, 2004)
comp['tr'] = [
    0.48 if x <= 1978 else (
        0.46 if 1979 <= x <= 1986 else (
            0.4 if x == 1987 else (
                0.34 if 1988 <= x <= 1992 else (
                    0.35
                )
            )
        )
    ) for x in comp['fyear']
]
def create_tb_1(row):
    if row['ib'] == 0:
        return np.nan
    if (np.nansum([row['txfo'], row['txfed']]) > 0 or row['txt'] >  row['txdi']) \
            and (row['ib'] <= 0):
        return 1    
    if pd.isnull(row['txfo']) or pd.isnull(row['txfed']):
        return ((row['txt'] - row['txdi']) / row['tr']) / row['ib']
    else:
        return (np.nansum([row['txfo'], row['txfed']]) / row['tr']) / row['ib']
comp['tb_1'] = comp.apply(lambda x: create_tb_1(x), axis = 1)

# Prep for Mohanram (2005) score
comp['roa'] = comp['ni'] / ((comp['at'] + comp['at_lag1'])/2).replace({ 0 : np.nan })
def create_cfroa(row):
    if (row['at'] + row['at_lag1']) == 0:
        return np.nan
    if pd.isnull(row['oancf']):
        return (row['ib'] + row['dp']) / ((row['at'] + row['at_lag1'])/2)
    else:
        return row['oancf'] / ((row['at'] + row['at_lag1'])/2)
comp['cfroa'] = comp.apply(lambda x: create_cfroa(x), axis = 1)
comp['xrdint'] = comp['xrd'] / ((comp['at'] + comp['at_lag1'])/2).replace({ 0 : np.nan })
comp['capxint'] = comp['capx'] / ((comp['at'] + comp['at_lag1'])/2).replace({ 0 : np.nan })
comp['xadint'] = comp['xad'] / ((comp['at'] + comp['at_lag1'])/2).replace({ 0 : np.nan })

# Drop observations that do not have lagged observations to create other variables
comp.loc[comp['count'] == 1, 
         ['chadv', 'agr', 'invest', 'gma', 'chcsho', 
          'lgr', 'egr', 'chpm', 'chinv', 'hire',
          'cf', 'acc', 'pctacc', 'absacc', 'spi', 
          'sgr', 'pchsale_pchinvt', 'pchsale_pchrect',
          'pchgm_pchsale', 'pchsale_pchxsga', 'pchcapx',
          'ps', 'roa', 'cfroa', 'xrdint', 'capxint',
          'xadint', 'divi', 'divo', 'obklg', 'chobklg',
          'grltnoa', 'chdrc', 'rd', 'pchdepr', 'grGW',
          'pchcurrat', 'pchquick', 'pchsaleinv', 'roe', 'operprof']
        ] = np.nan
comp.loc[comp['count'] < 3, 
         ['chato', 'grcapx']
        ] = np.nan

## Industry adjustments for annual variables
# Sum each industry's sales figures
comp['indsale'] = comp[['sale','sic2','fyear']].groupby(['sic2','fyear']).transform('sum')
# De-mean by industry 
cols_to_demean = ['chpm','chato','hire','bm','pchcapx','tb_1','cfp','mve_f']
cols_mean = comp[cols_to_demean + ['sic2','fyear']].groupby(['sic2','fyear']).transform('mean')
cols_demeaned = ['chpmia','chatoia','chempia','bm_ia','pchcapx_ia','tb','cfp_ia','mve_ia']
comp[cols_demeaned] = comp[cols_to_demean] - cols_mean


# Create industry medians
df_indmd = comp.groupby(
    ['fyear', 'sic2']
)[[
    'roa','cfroa','xrdint','capxint','xadint'
]].median(
).reset_index(
    drop = False
).rename(
    columns = {'roa' : 'md_roa',
               'cfroa' : 'md_cfroa',
               'xrdint' : 'md_xrdint',
               'capxint' : 'md_capxint',
               'xadint' : 'md_xadint'}
)
comp = pd.merge(
    left = comp,
    right = df_indmd,
    how='left',
    left_on=['fyear','sic2'],
    right_on=['fyear','sic2']
).sort_values(['gvkey','datadate'])

# Continue creating Mohanram (2005) score
comp['m1'] = comp.apply(lambda row: 1 if row['roa'] > row['md_roa'] else 0, axis = 1)
comp['m2'] = comp.apply(lambda row: 1 if row['cfroa'] > row['md_cfroa'] else 0, axis = 1)
comp['m3'] = comp.apply(lambda row: 1 if row['oancf'] > row['ni'] else 0, axis = 1)
comp['m4'] = comp.apply(lambda row: 1 if row['xrdint'] > row['md_xrdint'] else 0, axis = 1)
comp['m5'] = comp.apply(lambda row: 1 if row['capxint'] > row['md_capxint'] else 0, axis = 1)
comp['m6'] = comp.apply(lambda row: 1 if row['xadint'] > row['md_xadint'] else 0, axis = 1)

## Connect to WRDS
# Download credit rating for Mohanram (2005) score
adsprate = conn.raw_sql("""
                    SELECT 
                        gvkey, datadate, splticrm

                    FROM comp.adsprate
                    
                    WHERE datadate >= '01/01/1975'
                    """)

comp['datadate'] = pd.to_datetime(comp['datadate'])
adsprate[['gvkey']] = adsprate[['gvkey']].astype(int)
adsprate['datadate'] = pd.to_datetime(adsprate['datadate'])
# Merge credit to comp
comp = pd.merge(
    left= comp,
    right= adsprate,
    how='left',
    left_on=['gvkey','datadate'],
    right_on=['gvkey','datadate']
).sort_values(['gvkey','datadate'])

# Assign value to credit rating
convert_credit = {
    'D' : 1,
    'C' : 2,
    'CC': 3,
    'CCC-': 4,
    'CCC' : 5,
    'CCC+' : 6,
    'B-' : 7,
    'B' : 8,
    'B+' : 9,
    'BB-' : 10,
    'BB' : 11,
    'BB+' : 12,
    'BBB-' : 13,
    'BBB' : 14,
    'BBB+' : 15,
    'A-' : 16,
    'A' : 17,
    'A+' : 18,
    'AA-' :19,
    'AA' : 20,
    'AA+' : 21,
    'AAA' : 22           
}
comp['credrat'] = comp['splticrm'].map(convert_credit)

# Credit downgrade indicator
comp['credrat_lag1'] = comp.groupby('gvkey')['credrat'].shift(1)
def create_credrat_dwn(row):
    if row['count'] == 1:
        return 0
    if row['credrat'] < row['credrat_lag1']:
        return 1
    else:
        return 0
comp['credrat_dwn'] = comp.apply(lambda x: create_credrat_dwn(x), axis = 1)
comp.drop(columns = ['credrat_lag1'], inplace = True)



# Save file
comp.to_csv("../data/comp.csv", index=False)

CPU times: user 6min 28s, sys: 31.7 s, total: 7min
Wall time: 7min 29s


### Load features

In [4]:
# Load comp file
comp = pd.read_csv("../data/comp.csv")

## CRSP link table

In [3]:
%%time
# Orgcap measure (Eisfeldt and Papanikolaou, 2013, JF)
# Hand type CPI table from Bureau of Labor Statistics
cpi = [[2020, 258.811],
       [2019, 255.657],
       [2018, 251.107],
       [2017, 245.120],
       [2016, 240.007],
       [2015, 237.017],
       [2014, 236.736],
       [2013, 232.957], 
       [2012, 229.594],
       [2011, 224.939],
       [2010, 218.056],
       [2009, 214.537],
       [2008, 215.303],
       [2007, 207.342],
       [2006, 201.6],
       [2005, 195.3],
       [2004, 188.9],
       [2003, 183.96],
       [2002, 179.88],
       [2001, 177.1],
       [2000, 172.2],
       [1999, 166.6],
       [1998, 163.00],
       [1997, 160.5],
       [1996, 156.9],
       [1995, 152.4],
       [1994, 148.2],
       [1993, 144.5],
       [1992, 140.3],
       [1991, 136.2],
       [1990, 130.7],
       [1989, 124.00],
       [1988, 118.3],
       [1987, 113.6],
       [1986, 109.6],
       [1985, 107.6],
       [1984, 103.9],
       [1983, 99.6],
       [1982, 96.5],
       [1981, 90.9],
       [1980, 82.4],
       [1979, 72.6],
       [1978, 65.2],
       [1977, 60.6],
       [1976, 56.9],
       [1975, 53.8],
       [1974, 49.3]]
cpi = pd.DataFrame(cpi, columns = ['year','cpi'])
comp = pd.merge(
    left= comp,
    right= cpi,
    how='left',
    left_on=['fyear'],
    right_on=['year']
).sort_values(
    ['gvkey','datadate']
).drop(columns = 'year')

# Finish up organization capital measure (Eisfeldt and Papanikolaou, 2013, JF)
comp['avgat'] = (comp['at'] + comp['at_lag1'])/2
comp['orgcap_1'] = np.nan

for i in range(0, len(comp)):
    if comp.loc[i, 'count'] == 1:
        comp.loc[i, 'orgcap_1'] = ( (comp.loc[i, 'xsga'] / comp.loc[i, 'cpi']) / (0.1 + 0.15) )
    else:
        comp.loc[i, 'orgcap_1'] = ( comp.loc[i-1, 'orgcap_1'] ) * (1 - 0.15) \
                                + (comp.loc[i, 'xsga'] / comp.loc[i, 'cpi'])

comp['orgcap'] = comp['orgcap_1'] / comp['avgat'].replace({ 0 : np.nan })
comp.loc[comp['count'] == 1, 'orgcap'] = np.nan   

CPU times: user 22min 9s, sys: 10min 5s, total: 32min 14s
Wall time: 6min 28s


In [37]:
# Create CRSP link table
lnk = conn.raw_sql("""
                    SELECT 
                        gvkey, linkprim, liid, linktype, lpermno as permno, lpermco as permco, USEDFLAG, linkdt, linkenddt

                    FROM crsp.ccmxpf_linktable
                    
                    WHERE linktype in ('LU', 'LC', 'LD', 'LF', 'LN', 'LO', 'LS', 'LX')
                    """)

#                     AND (linkdt <= '01/01/2021')
#                     AND (linkenddt >= '01/01/1950')

lnk[['gvkey']] = lnk[['gvkey']].astype(int)
lnk['linkdt'] = pd.to_datetime(lnk['linkdt'])
lnk['linkenddt'] = pd.to_datetime(lnk['linkenddt'])
lnk.sort_values(['gvkey','linkdt'], inplace = True)

In [40]:
lnk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35746 entries, 0 to 35745
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   gvkey      35746 non-null  int64         
 1   linkprim   35746 non-null  object        
 2   liid       35746 non-null  object        
 3   linktype   35746 non-null  object        
 4   permno     35746 non-null  float64       
 5   permco     35746 non-null  float64       
 6   usedflag   35746 non-null  float64       
 7   linkdt     35746 non-null  datetime64[ns]
 8   linkenddt  27125 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(3), int64(1), object(3)
memory usage: 2.7+ MB


In [69]:
# Create temp table
pd.merge(
    left=comp,
    right=lnk[['gvkey','permno']],
    how='left',
    left_on=['gvkey'],
    right_on=['gvkey']
)
lnk[['gvkey','permno']]

,gvkey,permno
0,1000,25881.0
1,1001,10015.0
2,1002,10023.0
3,1003,10031.0
4,1004,54594.0
...,...,...
35741,345189,14299.0
35742,345189,14299.0
35743,345556,16069.0
35744,345920,20194.0


In [12]:
lnk = pd.read_csv("../data/lnk.csv")
lnk

,gvkey,linkprim,liid,linktype,lpermno,lpermco,USEDFLAG,linkdt,linkenddt
0,1000,P,01,LU,25881,23369,1,19701113,19780630
1,1001,P,01,LU,10015,6398,1,19830920,19860731
2,1002,C,01,LC,10023,22159,1,19721214,19730605
3,1003,C,01,LU,10031,6672,1,19831207,19890816
4,1004,P,01,LU,54594,20000,1,19720424,E
...,...,...,...,...,...,...,...,...,...
32160,315887,P,01,LC,14344,54677,1,20131212,E
32161,316056,P,01,LC,14297,54641,1,20131202,E
32162,317260,P,01,LC,14756,54928,1,20140612,20190417
32163,317264,P,01,LC,14642,54852,1,20140508,E


In [44]:
# Check table
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

check = pd.read_csv("../data/check.csv")

In [45]:
check

,permno,cnum,gvkey,datadate,fyear,cik,sic2,sic,naics,sale,revt,cogs,xsga,dp,xrd,xad,ib,ebitda,ebit,nopi,spi,pi,txp,ni,txfed,txfo,txt,xint,capx,oancf,dvt,ob,gdwlia,gdwlip,gwo,rect,act,che,ppegt,invt,at,aco,intan,ao,ppent,gdwl,fatb,fatl,lct,dlc,dltt,lt,dm,dcvt,cshrc,dcpstk,pstk,ap,lco,lo,drc,drlt,txdi,ceq,scstkc,emp,csho,prcc_f,mve_f,count,dr,dc,xint0,xsga0,bm,ep,cashpr,dy,lev,sp,roic,rd_sale,rd_mve,agr,gma,chcsho,lgr,acc,pctacc,cfp,absacc,age,chinv,spii,cf,hire,sgr,chpm,chato,pchsale_pchinvt,pchsale_pchrect,pchgm_pchsale,pchsale_pchxsga,depr,pchdepr,chadv,invest,egr,pchcapx,grcapx,grGW,woGW,tang,sin,currat,pchcurrat,quick,pchquick,salecash,salerec,saleinv,pchsaleinv,cashdebt,realestate,divi,divo,obklg,chobklg,securedind,secured,convind,conv,grltnoa,chdrc,rd,rdbias,roe,operprof,ps,tr,tb_1,roa,cfroa,xrdint,capxint,xadint,b,chpmia,chatoia,indsale,chempia,bm_ia,pchcapx_ia,tb,cfp_ia,mve_ia,md_roa,md_cfroa,md_xrdint,md_capxint,md_xadint,m1,m2,m3,m4,m5,m6,splticrm,cpi,credrat,credrat_dwn,orgcap_1,avgat,orgcap
0,14717,234255,170994,20141231,2014,1367311.0,51,5171,424710.0,28.281,28.281,8.040,11.931,4.333,NaN,NaN,-3.264,8.310,3.977,0.218,0.0000,1.402,0.000,-3.264,0.000,0.000,-0.855,2.793,12.285,9.161,0.0,0.0,NaN,NaN,NaN,4.064,11.121,4.691,62.336,0.000,99.127,2.366,0.000,31.813,56.193,0.000,NaN,NaN,33.049,23.250,25.250,102.905,25.250,0.000,NaN,0.000,0.0,7.388,2.411,44.606,0.000,0.000,-1.173,-3.778,NaN,0.014,55.045,1.7600,96.879200,3,0.000,0.000,2.793,11.931,-0.038997,-0.033691,4.903475,0.0,1.062199,0.291920,0.039805,NaN,NaN,0.138671,0.232508,0.015478,3.212411,-0.133472,-3.806679,0.094561,0.133472,3,0.000000,0,0.098409,0.750000,80.034384,4.827280,0.298291,NaN,80.025697,-23.037249,79.622097,0.077109,21.991408,NaN,0.078755,-1.101962,75.781250,4.805766,0.000000,0,0.379917,0,0.336500,-0.844706,0.336500,-0.844706,6.028779,6.958907,NaN,NaN,0.016790,NaN,0.0,0.0,0.0,0.0,1,1.000000,0,0.000000,-0.066720,0.000000,0.0,NaN,-0.088090,0.148895,3.0,0.35,1.000000,-0.035062,0.098409,NaN,0.131968,NaN,NaN,3.397370,0.068263,777946.962,0.593175,3.936523,73.250271,-3.119409,0.346851,-2816.316345,0.028880,0.070118,0.000000,0.035137,0.020911,0,1,1,0,1,0,NaN,236.736,NaN,0,0.117751,93.0910,0.001265
1,14717,234255,170994,20151231,2015,1367311.0,51,5171,424710.0,29.213,29.213,6.796,14.520,4.765,NaN,NaN,-24.968,7.897,3.132,-1.705,0.1271,4.314,0.000,-24.968,0.000,0.000,29.282,8.071,5.137,-5.192,0.0,0.0,NaN,NaN,NaN,8.989,11.359,1.821,67.099,0.000,73.334,0.549,0.000,5.784,56.191,0.000,NaN,NaN,14.103,3.283,53.693,100.335,53.693,0.000,NaN,0.000,0.0,4.791,6.029,32.429,0.000,0.000,1.856,-27.001,NaN,0.049,55.175,0.2487,13.722022,4,0.000,0.000,8.071,14.520,-1.967713,-1.819557,-3.250400,0.0,7.311969,2.128914,0.067638,NaN,NaN,-0.260202,0.226144,0.002362,-0.024974,-0.229339,-0.792054,-0.378370,0.229339,4,0.000000,1,-0.060211,2.500000,0.032955,-0.739275,0.034979,NaN,-1.178905,0.074550,-0.184043,0.084800,0.099739,NaN,0.048049,6.146903,-0.581848,31.106250,0.000000,0,0.522409,0,0.805431,1.393553,0.805431,1.393553,16.042284,3.249861,NaN,NaN,-0.198809,NaN,0.0,0.0,0.0,0.0,1,1.000000,0,0.000000,-0.105404,0.000000,0.0,NaN,6.608788,0.046056,4.0,0.35,1.000000,-0.289550,-0.060211,NaN,0.059573,NaN,NaN,-98.131100,0.434575,758357.707,2.471115,-1.588715,-6.015595,0.311681,-0.543531,-2409.744731,0.027032,0.070263,0.000000,0.027798,0.015715,0,0,1,0,1,0,NaN,237.017,NaN,0,0.161350,86.2305,0.001871
2,13543,379577,171002,20121231,2012,1237831.0,38,3841,339112.0,385.994,385.994,57.091,196.669,18.108,27.926,0.400,73.831,132.234,114.126,-0.140,NaN,114.653,0.523,73.831,40.338,0.345,40.822,NaN,24.684,76.519,0.0,0.0,NaN,NaN,NaN,58.601,360.110,212.400,146.747,62.310,447.133,26.799,24.957,0.977,61.089,15.372,8.770,0.000,39.508,0.000,0.000,60.631,0.000,0.000,NaN,0.000,0.0,12.547,26.438,13.409,0.000,0.000,-6.280,386.502,NaN,0.810,91.270,10.4900,957.422300,1,0.000,0.000,0.000,196.669,0.403690,0.077114,2.402492,0.0,0.063327,0.403160,0.486791,0.072348,0.029168,NaN,NaN,NaN,NaN,NaN,NaN,0.079922,NaN,1,NaN,1,NaN,NaN,N

cnum                0
gvkey               0
datadate            0
fyear               0
cik             40010
                ...  
credrat_dwn         0
cpi               291
avgat           30712
orgcap_1        84665
orgcap         110152
Length: 208, dtype: int64

In [46]:
check.describe()

,permno,gvkey,datadate,fyear,cik,sic2,sic,naics,sale,revt,cogs,xsga,dp,xrd,xad,ib,ebitda,ebit,nopi,spi,pi,txp,ni,txfed,txfo,txt,xint,capx,oancf,dvt,ob,gdwlia,gdwlip,gwo,rect,act,che,ppegt,invt,at,aco,intan,ao,ppent,gdwl,fatb,fatl,lct,dlc,dltt,lt,dm,dcvt,cshrc,dcpstk,pstk,ap,lco,lo,drc,drlt,txdi,ceq,scstkc,emp,csho,prcc_f,mve_f,count,dr,dc,xint0,xsga0,bm,ep,cashpr,dy,lev,sp,roic,rd_sale,rd_mve,agr,gma,chcsho,lgr,acc,pctacc,cfp,absacc,age,chinv,spii,cf,hire,sgr,chpm,chato,pchsale_pchinvt,pchsale_pchrect,pchgm_pchsale,pchsale_pchxsga,depr,pchdepr,chadv,invest,egr,pchcapx,grcapx,grGW,woGW,tang,sin,currat,pchcurrat,quick,pchquick,salecash,salerec,saleinv,pchsaleinv,cashdebt,realestate,divi,divo,obklg,chobklg,securedind,secured,convind,conv,grltnoa,chdrc,rd,rdbias,roe,operprof,ps,tr,tb_1,roa,cfroa,xrdint,capxint,xadint,b,chpmia,chatoia,indsale,chempia,bm_ia,pchcapx_ia,tb,cfp_ia,mve_ia,md_roa,md_cfroa,md_xrdint,md_capxint,md_xadint,m1,m2,m3,m4,m5,m6,cpi,credrat,credrat_dwn,orgcap_1,avgat,orgcap
count,260387.000000,260387.000000,2.603870e+05,260387.000000,2.445540e+05,260387.000000,260387.000000,251869.000000,259802.000000,259802.000000,259800.000000,215455.000000,248523.000000,126312.000000,93968.000000,260387.000000,253779.000000,251904.000000,259754.000000,228350.000000,259820.000000,234734.000000,260387.000000,194652.000000,197599.000000,260099.000000,227043.000000,257974.000000,187508.000000,259143.000000,73227.000000,9645.000000,9673.000000,0.0,2.547390e+05,2.522270e+05,2.597470e+05,231193.000000,254557.000000,2.603870e+05,238265.000000,234860.000000,2.597040e+05,254412.000000,175998.000000,127549.000000,114635.000000,2.577560e+05,259227.000000,259277.000000,2.597200e+05,192710.000000,236475.000000,65297.000000,236219.000000,260082.000000,2.574220e+05,238250.000000,2.600360e+05,85908.000000,90270.000000,223372.000000,260153.000000,7523.000000,244727.000000,260006.000000,260387.000000,2.600060e+05,260387.000000,91280.000000,236482.000000,260387.000000,260387.000000,259793.000000,259999.000000,257041.000000,258872.000000,2.593610e+05,259426.000000,2.511970e+05,123855.000000,126205.000000,239290.000000,238810.000000,2.388300e+05,238461.000000,223778.000000,223778.000000,241506.000000,223778.000000,260387.000000,233284.000000,260387.000000,223778.000000,235509.000000,234300.000000,233544.000000,218438.000000,179720.000000,226207.000000,234260.000000,192148.000000,241914.000000,220939.000000,80983.000000,227582.000000,239033.000000,225533.000000,206260.000000,239300.000000,260387.000000,244637.000000,260387.000000,249896.000000,228807.000000,248378.000000,227156.000000,257876.000000,249234.000000,195525.000000,177194.000000,247897.000000,100806.000000,239300.000000,239300.000000,67556.000000,65210.000000,260387.000000,150839.000000,260387.000000,192377.000000,182694.000000,82294.000000,239300.000000,93370.000000,239106.000000,238631.000000,239300.000000,260387.000000,231399.000000,239296.000000,234483.000000,116016.000000,238727.000000,85709.000000,21087.0,233544.000000,218438.000000,2.603870e+05,235509.000000,259793.000000,225533.000000,231399.000000,241506.000000,2.600060e+05,259476.000000,259476.000000,254449.000000,259476.000000,257297.000000,260387.000000,260387.000000,260387.000000,260387.000000,260387.000000,260387.000000,260195.000000,47270.000000,260387.000000,197722.000000,2.603860e+05,180317.000000
mean,59651.833072,38489.265904,1.997911e+07,1997.668597,6.995638e+05,46.441251,4687.048136,397881.842760,2287.491893,2287.491893,1545.150697,385.716651,134.625696,90.558072,63.709467,135.183699,426.546896,297.981424,4.530442,-0.013363,207.051183,28.290101,136.328799,27.555240,22.541725,65.783420,76.699328,170.339979,407.293190,67.825236,607.174944,-193.610486,-250.219121,NaN,2.318161e+03,3.330820e+03,8.308126e+02,2059.103471,411.140830,7.316512e+03,93.552928,586.698217,7.583026e+02,1070.147683,482.332468,248.607010,74.798695,2.666484e+03,706.376461,1121.070778,5.936070e+03,187.989798,22.269205,0.451792,29.781620,28.130764,2.2805

In [47]:
comp.describe()

,gvkey,fyear,cik,sic2,sic,naics,sale,revt,cogs,xsga,dp,xrd,xad,ib,ebitda,ebit,nopi,spi,pi,txp,ni,txfed,txfo,txt,xint,capx,oancf,dvt,ob,gdwlia,gdwlip,gwo,rect,act,che,ppegt,invt,at,aco,intan,ao,ppent,gdwl,fatb,fatl,lct,dlc,dltt,lt,dm,dcvt,cshrc,dcpstk,pstk,ap,lco,lo,drc,drlt,txdi,ceq,scstkc,emp,csho,prcc_f,mve_f,count,dr,dc,xint0,xsga0,at_lag1,lt_lag1,act_lag1,che_lag1,lct_lag1,dlc_lag1,txp_lag1,invt_lag1,emp_lag1,sale_lag1,rect_lag1,cogs_lag1,xsga_lag1,dp_lag1,ppent_lag1,xad_lag1,ppegt_lag1,ceq_lag1,capx_lag1,gdwl_lag1,dvt_lag1,ob_lag1,aco_lag1,intan_lag1,ao_lag1,ap_lag1,lco_lag1,lo_lag1,dr_lag1,...,capx_lag2,at_lag2,bm,ep,cashpr,dy,lev,sp,roic,rd_sale,rd_mve,agr,gma,chcsho,lgr,acc,pctacc,cfp,absacc,age,chinv,spii,cf,hire,sgr,chpm,chato,pchsale_pchinvt,pchsale_pchrect,pchgm_pchsale,pchsale_pchxsga,depr,pchdepr,chadv,invest,egr,pchcapx,grcapx,grGW,woGW,tang,sin,currat,pchcurrat,quick,pchquick,salecash,salerec,saleinv,pchsaleinv,cashdebt,realestate,divi,divo,obklg,chobklg,securedind,secured,convind,conv,grltnoa,chdrc,rd,rdbias,roe,operprof,ps,tr,tb_1,roa,cfroa,xrdint,capxint,xadint,indsale,chpmia,chatoia,chempia,bm_ia,pchcapx_ia,tb,cfp_ia,mve_ia,md_roa,md_cfroa,md_xrdint,md_capxint,md_xadint,m1,m2,m3,m4,m5,m6,credrat,credrat_dwn,cpi,avgat,orgcap_1,orgcap
count,333594.000000,333594.000000,2.935840e+05,333593.000000,333593.000000,324329.000000,332990.000000,332990.000000,332985.000000,274270.000000,320141.000000,151447.00000,109764.000000,333594.000000,325834.000000,324546.000000,332923.000000,290645.000000,333012.000000,303763.000000,333594.000000,242600.000000,244292.000000,333292.000000,293229.000000,330776.000000,250343.000000,331730.000000,81311.000000,12390.000000,12426.000000,0.0,3.258930e+05,3.243430e+05,3.328970e+05,297338.000000,327016.000000,3.335940e+05,307858.000000,306283.000000,3.328470e+05,326980.000000,241434.000000,169281.000000,155217.000000,3.308560e+05,332305.000000,3.324080e+05,3.328820e+05,257643.000000,305616.000000,71354.000000,305261.000000,332994.000000,3.304290e+05,307834.000000,3.331920e+05,134870.000000,140089.000000,285515.000000,333134.000000,8653.000000,293282.000000,332847.000000,333594.000000,3.328470e+05,333594.000000,141274.000000,305688.000000,333594.000000,333594.000000,3.028820e+05,3.022810e+05,253185.000000,302271.000000,256889.000000,301732.000000,275797.000000,296795.000000,267287.000000,302327.000000,2.958050e+05,302319.000000,248293.000000,290565.000000,296794.000000,99916.000000,270332.000000,302511.000000,300215.000000,216969.000000,301476.000000,75108.000000,279444.000000,277410.000000,3.022190e+05,2.999650e+05,279434.000000,3.025560e+05,124871.000000,...,271675.000000,2.741460e+05,3.323790e+05,3.327670e+05,3.242060e+05,331089.000000,3.320920e+05,3.321760e+05,3.226330e+05,145209.000000,1.511960e+05,302211.000000,301707.000000,3.020090e+05,301822.000000,285185.000000,285516.000000,3.031740e+05,285185.000000,333594.000000,295734.000000,333594.000000,285185.000000,296960.000000,284968.000000,282670.000000,273282.000000,211881.000000,273467.000000,284876.000000,229713.000000,304800.000000,272462.000000,93267.000000,288631.000000,302276.000000,277165.000000,251543.000000,302882.000000,333594.000000,313849.000000,333594.000000,321324.000000,251527.000000,319429.000000,249452.000000,325417.000000,312361.000000,232673.000000,207710.000000,289929.000000,132215.000000,302882.000000,302882.00000,74836.000000,71971.000000,333594.000000,186811.000000,333594.000000,232435.000000,237455.000000,124196.000000,271118.000000,105198.000000,302392.000000,301894.000000,302882.000000,333594.000000,287624.000000,302551.000000,297409.000000,137892.000000,301965.000000,99107.000000,3.335930e+05,2.826700e+05,2.732820e+05,2.969600e+05,3.323780e+05,2.771650e+05,2.876230e+05,3.031730e+05,3.328460e+05,332628.000000,332628.000000,326390.000000,332628.000000,330001.000000,333594.000000,333594.000000,333594.000000,333594.000000,333594.000000,333594.000000,46568.000000,333594.000000,333303.000000,3.028820e+05,248929.0

In [52]:
comp

,cnum,gvkey,datadate,fyear,cik,sic2,sic,naics,sale,revt,cogs,xsga,dp,xrd,xad,ib,ebitda,ebit,nopi,spi,pi,txp,ni,txfed,txfo,txt,xint,capx,oancf,dvt,ob,gdwlia,gdwlip,gwo,rect,act,che,ppegt,invt,at,aco,intan,ao,ppent,gdwl,fatb,fatl,lct,dlc,dltt,lt,dm,dcvt,cshrc,dcpstk,pstk,ap,lco,lo,drc,drlt,txdi,ceq,scstkc,emp,csho,prcc_f,mve_f,count,dr,dc,xint0,xsga0,at_lag1,lt_lag1,act_lag1,che_lag1,lct_lag1,dlc_lag1,txp_lag1,invt_lag1,emp_lag1,sale_lag1,rect_lag1,cogs_lag1,xsga_lag1,dp_lag1,ppent_lag1,xad_lag1,ppegt_lag1,ceq_lag1,capx_lag1,gdwl_lag1,dvt_lag1,ob_lag1,aco_lag1,intan_lag1,ao_lag1,ap_lag1,lco_lag1,...,at_lag2,bm,ep,cashpr,dy,lev,sp,roic,rd_sale,rd_mve,agr,gma,chcsho,lgr,acc,pctacc,cfp,absacc,age,chinv,spii,cf,hire,sgr,chpm,chato,pchsale_pchinvt,pchsale_pchrect,pchgm_pchsale,pchsale_pchxsga,depr,pchdepr,chadv,invest,egr,pchcapx,grcapx,grGW,woGW,tang,sin,currat,pchcurrat,quick,pchquick,salecash,salerec,saleinv,pchsaleinv,cashdebt,realestate,divi,divo,obklg,chobklg,securedind,secured,convind,conv,grltnoa,chdrc,rd,rdbias,roe,operprof,ps,tr,tb_1,roa,cfroa,xrdint,capxint,xadint,indsale,chpmia,chatoia,chempia,bm_ia,pchcapx_ia,tb,cfp_ia,mve_ia,md_roa,md_cfroa,md_xrdint,md_capxint,md_xadint,m1,m2,m3,m4,m5,m6,splticrm,credrat,credrat_dwn,cpi,avgat,orgcap_1,orgcap
0,000032,1000,1975-12-31,1975.0,NaN,30.0,3089.0,NaN,51.192,51.192,35.237,9.591,1.029,NaN,NaN,2.284,6.364,5.335,-0.304,NaN,4.224,0.294,2.284,0.955,0.000,1.940,0.807,3.109,NaN,0.124,6.3,NaN,NaN,NaN,8.193,16.366,1.679,12.700,6.227,23.905,0.267,0.000,0.123,7.263,NaN,NaN,NaN,6.224,1.251,4.592,11.596,NaN,2.50,NaN,2.50,2.069,2.554,2.125,0.000,NaN,NaN,0.689,10.240,NaN,1.054,2.098,4.3750,9.178750,1,NaN,2.50,0.807,9.591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.115620,0.248836,-6.035884,0.013509,1.263353,5.577230,0.279754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.141677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.620325,0,2.629499,NaN,1.629017,NaN,30.489577,6.248261,8.220973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1,0.544425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.48,0.871096,NaN,NaN,NaN,NaN,NaN,22997.699,NaN,NaN,NaN,-0.779905,NaN,-0.418439,NaN,-87.532281,0.057906,0.080100,0.016330,0.043252,0.010198,0,0,0,0,0,0,NaN,NaN,0,53.800,NaN,0.713086,NaN
1,000032,1000,1976-12-31,1976.0,NaN,30.0,3089.0,NaN,66.414,66.414,45.277,12.396,1.598,NaN,NaN,3.434,8.741,7.143,0.016,0.000000,6.174,1.135,3.434,2.009,0.000,2.740,0.985,7.023,NaN,0.094,10.8,NaN,NaN,NaN,9.207,21.647,2.090,23.003,9.919,38.586,0.431,0.000,0.313,16.440,NaN,NaN,NaN,7.907,0.350,13.583,22.911,NaN,2.25,NaN,2.25,1.569,3.937,2.485,0.000,NaN,NaN,0.237,14.106,NaN,1.460,2.207,5.7500,12.690250,2,NaN,2.25,0.985,12.396,23.905,11.596,16.366,1.679,6.224,1.251,0.294,6.227,1.054,51.192,8.193,35.237,9.591,1.029,7.263,NaN,12.700,10.240,3.109,NaN,0.124,6.3,0.267,0.000,0.123,2.554,2.125,...,NaN,1.111562,0.270601,-5.891268,0.007407,1.805402,5.233467,0.204054,NaN,NaN,0.614139,0.884208,0.051954,0.975768,0.048935,0.445253,0.150115,0.048935,2,0.118161,0,0.060969,0.385199,0.297351,0.007090,NaN,-0.295551,0.173587,0.027437,0.004889,0.097202,-0.313919,NaN,0.585442,0.377539,1.258926,NaN,0.0,0,0.593327,0,2.737701,0.041149,1.483243,-0.089486,31.777033,7.213425,6.695635,-0.185542,0.291651,NaN,0.0,0.0,0.345650,0.144021,0,NaN,1,0.165648,0.350931,NaN,0.0,NaN,0.335352,0.757422,3.0,0.48,1.218817,0.109904,0.161047,NaN,0.224768,NaN,25132.294,0.006186,NaN,0.345367,-0.337167,0.628603,-0.955842,0.031340,-94.651439,0.068410,0.107161,0.017049,0.055574,0.018593,1,1,0,0,1,0,NaN,NaN,0,56.900,31.2455,0.823979,0.026371
2,000032,1000,1977-12-31,1977.0,NaN,30.0,3089.0,NaN,77.946,77.946,56.959,14.977,2.507,NaN,NaN,1.928,6.010,3.503,0.502,0.022055,3.263,0.000,1.928,0.904,0.000,1.335,1.653,3.478,NaN,0.316,7.7,NaN,NaN,NaN,12.434,23.548,1.303,27.354,9.129,44.025,0.682,0.000,0.429,19.917,NaN,NaN,NaN,8.236,0.533,18.116,28.149,NaN,0.00,NaN,0.00,0.000,4.012,3.691,0.000,NaN,NaN,0.170,15.876,NaN,1.741,2.226,9.

In [55]:
comp.isnull().sum()

cnum                0
gvkey               0
datadate            0
fyear               0
cik             40010
                ...  
credrat_dwn         0
cpi               291
avgat           30712
orgcap_1        84665
orgcap         110152
Length: 208, dtype: int64